In [1]:
from collections import OrderedDict

import torch
from torch import nn


class IntermediateLayerGetter(nn.ModuleDict):
    """
    Module wrapper that returns intermediate laye
    \rs from a model
    It has a strong assumption that the modules have been registered
    into the model in the same order as they are used.
    This means that one should **not** reuse the same nn.Module
    twice in the forward if you want this to work.
    Additionally, it is only able to query submodules that are directly
    assigned to the model. So if `model` is passed, `model.feature1` can
    be returned, but not `model.feature1.layer2`.
    Arguments:
        model (nn.Module): model on which we will extract the features
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
    Examples::
        >>> m = torchvision.models.resnet18(pretrained=True)
        >>> # extract layer1 and layer3, giving as names `feat1` and feat2`
        >>> new_m = torchvision.models._utils.IntermediateLayerGetter(m,
        >>>     {'layer1': 'feat1', 'layer3': 'feat2'})
        >>> out = new_m(torch.rand(1, 3, 224, 224))
        >>> print([(k, v.shape) for k, v in out.items()])
        >>>     [('feat1', torch.Size([1, 64, 56, 56])),
        >>>      ('feat2', torch.Size([1, 256, 14, 14]))]
    """
    def __init__(self, model, return_layers):
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")

        orig_return_layers = return_layers
        return_layers = {k: v for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super(IntermediateLayerGetter, self).__init__(layers)
        self.return_layers = orig_return_layers
        #self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.named_children():
        #    print(name)
            out[name] = module(x)
            x= out[name]
        
        return out

In [2]:
import torch.nn as nn
#from .utils import load_state_dict_from_url



def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet2(nn.Module):

    def __init__(self, block, layers, num_classes=2, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet2, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        self.inplanes2 = 1024
        self.dilation2 = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer11 = self._make_layer(block, 64, 3,flag=True)
        self.layer22 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer33 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        
        self.layer1 = self._make_layer2(block, 64, layers[0])

        self.layer2 = self._make_layer2(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer2(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer2(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.ada_avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.ada_maxpool = nn.AdaptiveMaxPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.fc2 = nn.Linear(2, 1)
        self.sigmoid=nn.Sigmoid()
        self.softmax=nn.Softmax(dim=1)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False,flag=False):
        if flag:
            self.dilation2=self.dilation
            self.inplanes2=self.inplanes
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)
    def _make_layer2(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation2 = self.dilation2
        if dilate:
            self.dilation2 *= stride
            stride = 1
        if stride != 1 or self.inplanes2 != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes2, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes2, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation2, norm_layer))
        self.inplanes2 = planes * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.inplanes2, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        y = self.maxpool(x)
        y = self.layer1(y)
        y = self.layer2(y)
        y = self.layer3(y)
        y = self.layer4(y)
        y = self.ada_avgpool(y)
        y = y.reshape(y.size(0), -1)
        y = self.fc(y)
        y = self.softmax(y)
        
        x = self.layer11(x)
        x = self.layer22(x)
        x = self.layer33(x)
        x = self.ada_maxpool(x)#所以这里都是均值 
        
        x = torch.mean(x,(1))

        x = x.view(x.size(0))
        x = self.sigmoid(x)

        out = OrderedDict()
        out['x'] = x
        out['y'] = y
        
        return out


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet2(block, layers, **kwargs)
    return model



def resnet101_(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>'_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 9, 3], pretrained, progress,
                   **kwargs)


In [3]:
import numpy as np
import torch
from torchvision.models import resnet101
import torch.nn as nn
import os
import cv2
import random
import torch.utils.model_zoo as model_zoo
from copy import deepcopy

from matplotlib import cm
import scipy.misc as misc
import matplotlib
import imageio

def get_jet():
    colormap_int = np.zeros((256, 3), np.uint8)
    colormap_float = np.zeros((256, 3), np.float)

    for i in range(0, 256, 1):
        colormap_float[255-i, 0] = cm.jet(i)[0]
        colormap_float[255-i, 1] = cm.jet(i)[1]
        colormap_float[255-i, 2] = cm.jet(i)[2]

        colormap_int[255-i, 0] = np.int_(np.round(cm.jet(i)[0] * 255.0))
        colormap_int[255-i, 1] = np.int_(np.round(cm.jet(i)[1] * 255.0))
        colormap_int[255-i, 2] = np.int_(np.round(cm.jet(i)[2] * 255.0))
        #colormap_int = colormap_int[::-1]
    return colormap_int




class ban_tanh(nn.Module):
    def forward(self, x):
        x=torch.clamp(x,min=0)
        return torch.tanh(x)

# def load_data(img_path,label_path):
#     labels_list=open(label_path, 'r')
#     X,Y,Z=[],[],[]
#     i=1
#     for line in labels_list:
#         info=str.split(line,",") # info[1]label； info[2]图片文件名
#         X.append(np.concatenate([deepcopy (np.expand_dims(imageio.imread(info[1]),axis=2))for i in range(3)],axis=2))
#         Y.append(int(info[0]))
#         Z.append(str(info[2]))
#         i+=1
#         if i%100==0:
#             print(i)
# #         print(X)
# #         break;
#     return list(zip(X,Y,Z))

class FeatureExtractor(nn.Module):
    def __init__(self, submodule, extracted_layers):
        super(FeatureExtractor, self).__init__()
        self.submodule = submodule
        self.extracted_layers = extracted_layers

    def forward(self, x):
        outputs = []
        _name = []
        for name, module in self.submodule._modules.items():
            if name is "fc": x = x.view(x.size(0), -1)
            #print(name)
            _name.append(name)
            x = module(x)

            #if name in self.extracted_layers:
            outputs.append(x)
        return list(zip(_name,outputs))
class FeatureExtractor2(nn.Module):
    def __init__(self, submodule, extracted_layers):
        super(FeatureExtractor2, self).__init__()
        self.submodule = submodule
        self.extracted_layers = extracted_layers

    def forward(self, x):
        outputs = []
        for name, module in self.submodule._modules.items():
            if name is "fc": x = x.view(x.size(0), -1)
           # print(name)
            x = module(x)

            if name in self.extracted_layers:
                outputs.append(x)
        return outputs
    
def normalize_255_uchar(array):
    Max=np.max(array)
    Min=np.min(array)
    return (((array-Min)/(Max-Min))*255).astype(np.uint8)
learning_rate=0.0001

model=resnet101_().cuda()
#model=cam_model()
# criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
class_num = 2
channel_in = model.fc.in_features
model.fc = nn.Linear(channel_in,class_num).cuda()#注意加上这个cuda()

# return_layers = {'layer4': 'layer4'}
# model=IntermediateLayerGetter(model, return_layers=return_layers)

In [4]:
import csv,os


In [5]:
import csv,os
def get_csv():
#     class_path  =  '/media/students/000FF23E00059C2D/tq/2019/8/20/Train_c1/expert1/'
#     class_path3  = '/media/students/000FF23E00059C2D/tq/2019/8/20/img/'
    class_path  =  'F:\\Ruiveen2\\Data\\Train_c1\\expert1\\'
    class_path3  = 'F:\\Ruiveen2\\Data\\img\\'
    blanket=set()
    #############case3
#     blanket.add(6)
#     blanket.add(21)
#     blanket.add(27)
#################case1
#     blanket.add(8)
#     blanket.add(9)
#     blanket.add(2)
###############case2
#     blanket.add(5)
#     blanket.add(15)
#     blanket.add(18)
###############case4
    blanket.add(7)
    blanket.add(29)
    blanket.add(34)

#     if 2 in blanket:
#         print("fdsfdsfds")
    with open('my_heat4.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        i=0
        for img_name in os.listdir(class_path):
            info=str.split(img_name,"_")
            
#             print(img_name)
#             break
           # print(info[0])
            if int(info[0]) not in blanket:
                class_path2=class_path+img_name+"\\"
#                 print(class_path2)
#                 break
                for img_name2 in os.listdir(class_path2):
                    img_name3=info[1]+"_"+img_name2
                   # print(img_name3)
                    img_path2= class_path2+ img_name2
                    img_path = class_path3+info[1]+"\\"+ img_name2
#                     print(img_path)
#                     print(img_path2)
#                     break
                    temp=[2]
                    temp.append(img_path)
                    temp.append(img_name3)
                    temp.append(img_path2)
                    temp.append("")
                    f_csv.writerow(temp)
                   # i=i+1
#                     if i%100==0:
#                         print(i)
#                     break
             #   break
get_csv()


In [6]:
#get_csv()

In [7]:
def load_data(label_path):
    labels_list=open(label_path, 'r')
    X,Y,Z=[],[],[]
    i=1
    for line in labels_list:
        info=str.split(line,",") 
        X.append(np.expand_dims(imageio.imread(info[1]),axis=2))
        a=imageio.imread(info[3])
        b=np.max(a)
        if b!=0:
            b=1
        Y.append(b)
        Z.append(str(info[2]))
        #break
#         print(X[0].shape)
#         print(X[0].shape)
        i+=1
        if i%100==0:
            print(i)
#             if i==500:
#                 break
    return list(zip(X,Y,Z))

In [9]:
data=load_data("my_heat4.csv")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [10]:
mmpp=87685435454

In [11]:
model.load_state_dict(torch.load("com_heat3.pkl"))

<All keys matched successfully>

In [ ]:
learning_rate=0.0001

criterion2 = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

batch_size=2
num_epochs=1000
import random
for epoch in range(num_epochs):
    count=0
    random.shuffle(data)
    X, Y,_= zip(*data)
    X=np.array(X).transpose((0,3,1,2))
    Y=np.array(Y)
    print(X.shape)
    mm1=0
    mm2=0
    mm=0
    
#     incorrect = torch.zeros(1).squeeze().cuda()
#     total = torch.zeros(1).squeeze().cuda()
    
    for i in range(0,X.shape[0],batch_size):
        if i+batch_size>=X.shape[0]: break
        batch=torch.from_numpy(X[i:i+batch_size]).cuda().float()
        labels=torch.from_numpy(Y[i:i+batch_size]).cuda().float()
        labels2=torch.from_numpy(Y[i:i+batch_size]).cuda().long()
        outputs=model(batch)#
#         print(labels.shape)
#         print(outputs.shape)
#         print(labels)
#         print(outputs)

        loss1 = criterion(outputs['x'], labels)
        loss2 = criterion2(outputs['y'], labels2)
        loss = loss1+loss2
        mm1+=loss1.item()
        mm2+=loss2.item()
        mm+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print(outputs['y'].shape)
#         loss = criterion2(outputs['y'], labels2)
#         mm+=loss.item()
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        if count%100==0:
            print('Epoch [{}/{}], step[{}/{}],Loss: {:.20f}'.format(epoch + 1, num_epochs, count,X.shape[0]//batch_size,loss.item()))
            print(' loss: {:.20f}, cam_loss1: {:.20f},cnn_loss2: {:.20f}'.format(mm,mm1,mm2))
#             print(' loss: {:.20f}'.format(mm)
#             print(mm)
#             print(mm1)
#             print(mm2)
        count += 1
    if mm<mmpp:
        mmpp=mm
        print(' loss: {}, cam_loss1: {},cnn_loss2: {}'.format(mm,mm1,mm2))   
#         print(mm)
#         print()
        torch.save(model.state_dict(),"com_heat5.pkl")
    else:
        model.load_state_dict(torch.load("com_heat5.pkl"))
#     print(mm)
    print(' loss: {}, cam_loss1: {},cnn_loss2: {}'.format(mm,mm1,mm2))   
    print(mmpp)


(2304, 1, 1024, 512)
Epoch [1/1000], step[0/1152],Loss: 1.00797867774963378906
 loss: 1.00797867774963378906, cam_loss1: 0.69471430778503417969,cnn_loss2: 0.31326431035995483398
